In [4]:
from skimage.color import rgb2gray, rgb2hsv
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from skimage.filters import gaussian
import cv2

import numpy as np
import math

%matplotlib inline



In [5]:
def enhancement(crop_image):
     # Apply Gaussian blur
    blur = cv2.GaussianBlur(crop_image, (3,3), 0)
    
    # Change color-space from BGR -> HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    
    # Create a binary image with where white will be skin colors and rest is black
    #mask2 = cv2.inRange(hsv, np.array([0,0,0]), np.array([20,255,255]))
#     min_YCrCb = np.array([0,133,77],np.uint8)
#     max_YCrCb = np.array([235,173,127],np.uint8)
#     imageYCrCb = cv2.cvtColor(blur,cv2.COLOR_BGR2YCR_CB)
#     mask2 = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)
    # sensitivity = 20 
    # lower_bound = np.array([50 - sensitivity, 100, 60]) 
    # upper_bound = np.array([50 + sensitivity, 255, 255]) 
    mask2 = cv2.inRange(hsv, np.array([0, 20, 70], dtype = "uint8"), np.array([20, 255, 255], dtype = "uint8"))
    
    # Kernel for morphological transformation    
    kernel = np.ones((5,5))
    
    # Apply morphological transformations to filter out the background noise
    dilation = cv2.dilate(mask2, kernel, iterations = 2)
    erosion = cv2.erode(dilation, kernel, iterations = 1)    
       
    # Apply Gaussian Blur and Threshold
    mask = cv2.GaussianBlur(erosion, (3,3), 0)
    return mask     

In [6]:


#set initial values
flag = {"first": False, "operator": False, "second": False}
count_same_frames = 0
first, operator, second = "", "", ""
pred_text = ""
calc_text = ""
info = "Enter first number"        

# define a video capture object
vid = cv2.VideoCapture(0)

while (True):

    ret, frame = vid.read()
    dim = (1000, 800)
    frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    # Display the resulting frame
    frame = cv2.flip(frame, 1)
    # Get hand data from the rectangle sub window   
    cv2.rectangle(frame,(500,400),(990,790),(0, 0, 0),2)
    crop_image = frame[ 400:790,500:990]
    
    mask=enhancement(crop_image)
    
    
    

    try:
        ret, thresh = cv2.threshold(mask, 127, 255,0)
        contours, hierarchy = cv2.findContours(thresh,2,1)
        #new added 
        old_pred_text = pred_text
        if len(contours)!=0:
            #find contour of max area(hand)
            cnt = max(contours, key = lambda x: cv2.contourArea(x))
            # print(max(contours, key = lambda x: cv2.contourArea(x)))
         
            #drawing lines
            rows,cols = mask.shape[:2]
            [vx,vy,x,y] = cv2.fitLine(cnt, cv2.DIST_L2,0,0.01,0.01)
            lefty = int((-x*vy/vx) + y)
            righty = int(((cols-x)*vy/vx)+y)
            cv2.line(mask,(cols-1,righty),(0,lefty),(0,255,0),2)
            slope=(lefty-righty)/(0-(cols-1))
            # cv2.putText(frame, str(slope), (80, 200), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            #cv2.putText(frame, str(slope >= -0.1 and slope <=0.1), (80, 300), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            # cv2.putText(frame, str(slope >= 0.7 and slope <=1.3), (80, 300), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            # (x,y),radius = cv2.minEnclosingCircle(cnt)
            # center = (int(x),int(y))
            # radius = int(radius)
            # cv2.circle(frame,center,radius,(255,255,255),2)

            #get moment
            # M = cv2.moments(cnt)
            # cx = int(M['m10']/M['m00'])
            # cy = int(M['m01']/M['m00'])
            # cv2.putText(frame, str(cx), (80, 200), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            # cv2.putText(frame, str(cy), (80, 300), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            #premieter
            perimeter = cv2.arcLength(cnt,True)
            # cv2.putText(frame, str(perimeter), (80, 350), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            # cv2.putText(frame, str(perimeter>=2000), (80, 390), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            #area diameter
            #area = cv2.contourArea(cnt)
            #area hand
            areacnt = cv2.contourArea(cnt)
            equi_diameter = np.sqrt(4*areacnt/np.pi)
            # cv2.putText(frame, str(equi_diameter), (80, 390), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            # extreme points
            # leftmost = tuple(cnt[cnt[:,:,0].argmin()][0])
            # rightmost = tuple(cnt[cnt[:,:,0].argmax()][0])
            # topmost = tuple(cnt[cnt[:,:,1].argmin()][0])
            # bottommost = tuple(cnt[cnt[:,:,1].argmax()][0])
            # distance=np.sqrt( (leftmost[0]-rightmost[0])  ** 2 + (leftmost[1]-rightmost[1]) ** 2 )
            # cv2.putText(frame, str(leftmost), (80, 350), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            # cv2.putText(frame, str(rightmost), (80, 400), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            # cv2.putText(frame, str(distance), (80, 500), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            # #approx the contour a little
            
            #aspect ratio
            x,y,w,h = cv2.boundingRect(cnt)
            aspect_ratio = float(w)/h
            #cv2.putText(frame, str(aspect_ratio), (80, 500), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
            epsilon = 0.0005*cv2.arcLength(cnt,True)
            approx= cv2.approxPolyDP(cnt,epsilon,True)
            
            #make convex hull around hand
            hull = cv2.convexHull(cnt)
            #print(cnt)
            #define area of hull and area of hand
            areahull = cv2.contourArea(hull)
            

        #find the percentage of area not covered by hand in convex hull
            if areacnt!=0:
                arearatio=((areahull-areacnt)/areacnt)*100
            else :
                arearatio=0    
            #find the defects in convex hull with respect to hand
            hull = cv2.convexHull(approx, returnPoints=False)
            if len(hull)!=0:
                #print('hull')
                #print(hull)
                defects = cv2.convexityDefects(approx, hull)
               
                l=1   
          

                    #code for finding no. of defects due to fingers
                for i in range(defects.shape[0]):
                        s,e,f,d = defects[i,0]
                        start = tuple(approx[s][0])
                        end = tuple(approx[e][0])
                        far = tuple(approx[f][0])
                        pt= (100,180)


                        # find length of all sides of triangle
                        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                        s = (a+b+c)/2
                        area_triangle  = math.sqrt(s*(s-a)*(s-b)*(s-c))

                        #distance between point and convex hull
                        d=(2*area_triangle)/a
                        
                        # print('distance')
                        # print(d)
                        # print('arearatio')
                        # print(arearatio)
                        # print('areahull')
                        # print(areahull)
                        # print('areacnt')
                        # print(areacnt)
                        
                        #cv2.putText(frame, str(d), (80, 600), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
                        

                        # apply cosine rule here
                        angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14
                        #print(angle)        

                        
                        if angle <= 80:
                           l += 1
                           cv2.circle(crop_image,far,5,[255,255,255],-1)

                        if  flag['first'] == True and flag['operator'] == False and flag['second'] == False:
                            if(slope >= 0.7 and slope <=1.3):
                              sign='%' 
                              count_same_frames=31 
                            elif(arearatio>=0 and arearatio<=20 and l==1 ):
                                sign='*'
                                # cv2.putText(frame, 'فى الزفت الكونديشن', (80, 200), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
                            elif(arearatio>=0 and arearatio<=20 and l==2):
                                sign='/'
                            elif(arearatio>=45 ):
                                sign='^'    
                            elif(angle<=70  and l==2):
                              sign='+'

                            elif(angle>80 and angle <=100  ):
                              sign='-'  
                            cv2.putText(frame, str(sign), (80, 200), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )  
                             
                          
                        if(perimeter >= 3000):
                            l=6
                            # count_same_frames=31
                        # elif(aspect_ratio  >= 1.9 and aspect_ratio  <=2 ):
                        #     l=8       
                        elif(slope >= -0.1 and slope <=0.1):
                              if(aspect_ratio  >= 2.7 ):
                                 l=9    
                              else:
                                l=8

                            #   count_same_frames=31  
                           
                        elif(equi_diameter  >= 300):
                            l=7       
                          
                            # count_same_frames=31 
                             
                           

                              
                        # if(angle<=80):
                        #    sign='+'

                        # elif(angle>=100 and angle <=180):
                        #    sign='-'   
                        
                        # if(d>=0 and d <=10):
                        #    sign='*'

                        # elif(d>=100 and l==5 ):
                        #    sign='/'   
                        
                        #draw lines around hand
                        cv2.line(crop_image,start, end, [0,255,0], 2)
                        
                        cv2.circle(crop_image, start, 4, [255,0,0], -1)
                        cv2.circle(crop_image, end, 4, [0,0,255], -1)#[0,0,255]:red
                        cv2.circle(crop_image, far, 3, [0,255,0], -1)
               


    
               
                font = cv2.FONT_HERSHEY_SIMPLEX
                
                pred_text = l

                if old_pred_text == pred_text:
                        count_same_frames += 1
                else:
                        count_same_frames = 0
                
                if count_same_frames > 30 :
                    # count_same_frames = 0
                    # first, second, operator, pred_text, calc_text = '', '', '', '', ''
                    # flag['first'], flag['operator'], flag['second'], flag['clear'] = False, False, False, False
                    # info = "Enter first number"
                    if  flag['first'] == False and flag['operator'] == False and flag['second'] == False :
                         count_same_frames = 0
                         first, second, operator, pred_text, calc_text = pred_text, '', '', '', ''
                         info = "Enter the Sign"
                         flag['first'] = True 
                         flag['operator'] = False 
                         flag['second'] = False
                    elif flag['first'] == True and flag['operator'] == False and flag['second'] == False :
                         count_same_frames = 0
                         #operator will be changed later
                         second, operator, pred_text, calc_text = '', pred_text, '', ''
                         info = "Enter the second operand"
                         flag['first'] = True 
                         flag['operator'] = True  
                         flag['second'] = False
                    elif flag['first'] == True and flag['operator'] == True and flag['second'] == False :
                         count_same_frames = 0
                         second, pred_text, calc_text = pred_text,'', ''
                         if(sign=='+'):
                          result=first+ second
                         elif(sign=='-'):
                            result=first- second
                         elif(sign=='*'):
                            result=first* second  
                         elif(sign=='/'):
                            result=first/ second 
                         elif(sign=='^'):
                            result=pow(first, second) 
                         elif(sign=='%'):  
                            result=first % second    
                                  



                         #print(result)
                         info=str(first)+sign+str(second)
                         info += '='+ str(result)  
                         flag['first'] = True 
                         flag['operator'] = True  
                         flag['second'] =  True  
                #print(l)
                cv2.putText(frame, info, (30, 440), cv2.FONT_HERSHEY_TRIPLEX, 1, (0, 255, 255) )
                # if l==0:
                #     cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                if l==1:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

                elif l==2:
                    cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

                elif l==3:
                    cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif l==4:
                    cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)

                elif l==5:
                    cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif l==6:
                    cv2.putText(frame,'6',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)   
                elif l==7:
                    cv2.putText(frame,'7',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)     
                elif l==8:
                    cv2.putText(frame,'8',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif l==9:
                    cv2.putText(frame,'9',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)  
                # elif l==10:
                #     cv2.putText(frame,'10',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)          

                else :
                    cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)


       
        cv2.imshow('frame',frame)        
        cv2.imshow('mask',mask)  

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break 

        
    except:
        print('error and skip')
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()
